----

## STREAMLİT'İ ÇALIŞTIRMAK İÇİN

```BASH 
cd "C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\Final"
streamlit run streamlit_app.py


## Genel Metrics


In [10]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

FINAL_DIR = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\Final")

csv_candidates = list(FINAL_DIR.rglob("per_image_count_comparison.csv")) + \
                 list(FINAL_DIR.rglob("count_per_image.csv"))

if not csv_candidates:
    raise FileNotFoundError("Count CSV bulunamadı")

csv_path = sorted(csv_candidates, key=lambda p: p.stat().st_mtime, reverse=True)[0]
df = pd.read_csv(csv_path)

if "signed_error" not in df.columns:
    df["signed_error"] = df["pred_count"] - df["gt_count"]
df["abs_error"] = df["signed_error"].abs()

gt = df["gt_count"].values
pred = df["pred_count"].values
res = pred - gt

mae = np.mean(np.abs(res))
rmse = np.sqrt(np.mean(res**2))
bias = np.mean(res)
mape = np.mean(np.abs(res[gt > 0]) / gt[gt > 0])
exact = np.sum(res == 0)
n = len(df)

out_dir = FINAL_DIR / "outputs" / "final_viz"
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / "FINAL_EVIDENCE.png"

fig, axs = plt.subplots(2, 2, figsize=(14, 12))

# (A) GT vs Pred
axs[0,0].scatter(gt, pred, s=10, alpha=0.6)
mx = max(gt.max(), pred.max())
axs[0,0].plot([0, mx], [0, mx])
axs[0,0].set_title("GT vs Prediction (Ideal: y = x)")
axs[0,0].set_xlabel("GT Count")
axs[0,0].set_ylabel("Predicted Count")

# (B) Residual Histogram
axs[0,1].hist(res, bins=60)
axs[0,1].set_title("Residual Distribution (Pred − GT)")
axs[0,1].set_xlabel("Residual")
axs[0,1].set_ylabel("Frequency")

# (C) Residual vs GT
axs[1,0].scatter(gt, res, s=10, alpha=0.6)
axs[1,0].axhline(0)
axs[1,0].set_title("Residual vs GT (Bias Check)")
axs[1,0].set_xlabel("GT Count")
axs[1,0].set_ylabel("Residual")

# (D) Metrics Box
axs[1,1].axis("off")
text = (
    f"FINAL MODEL – COUNTING METRICS\n\n"
    f"Images          : {n}\n"
    f"MAE             : {mae:.3f}\n"
    f"RMSE            : {rmse:.3f}\n"
    f"MAPE            : {mape*100:.2f}%\n"
    f"Bias (Pred−GT)  : {bias:.3f}\n"
    f"Exact Matches   : {exact}\n"
)
axs[1,1].text(0.05, 0.95, text, va="top", fontsize=12)

plt.suptitle(
    "Head Counting Model — Final Configuration\n(conf=0.39 | DIoU=0.51 | imgsz=832)",
    fontsize=16
)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig(out_path, dpi=220)
plt.close()

print("KANIT GÖRSELİ ÜRETİLDİ:")
print(out_path)


KANIT GÖRSELİ ÜRETİLDİ:
C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\Final\outputs\final_viz\FINAL_EVIDENCE.png
